In [1]:
import re

In [2]:
sequence_length = 16

In [3]:
def txt_to_list(path, sequence_length):
    with open(path, 'r') as f:
        txt = str(f.read())
    txt_list = txt.split('\n')
    text_label_list = list()
    tmp = list()
    for line in txt_list:
        if line == '--------------------':
            text_label_list.append(tmp)
            tmp = list()
            continue
        if line == '':
            continue
        tmp.append(line)
    x = list()
    y = list()
    for text_label in text_label_list:
        text = text_label[0]
#         text_list = [i for i in text]
        label = text_label[2:]
        label_list = ['O' for i in range(len(text))]
        for i in label:
            entity = i.split('\t')
            b = int(entity[1])
            label_list[b] = 'B-{}'.format(entity[-1])
            for j in range(int(entity[1])+1, int(entity[2])):
                label_list[j] = 'I-{}'.format(entity[-1])
        for i, j in enumerate(text):
            if j == '，' or j == '。' or j == '？':
                label_list[i] = j
        
        text_list = re.split('\uff0c|\u3002|\uff1f', text)
        for sentence in text_list:
            x.append([i for i in sentence])
            
        tmp = list()
        for i in label_list:
            if i == '，' or i == '。' or i == '？':
                y.append(tmp)
                tmp = list()
            else:
                tmp.append(i)
#         while len(text_list) > sequence_length:            
#             x.append(text_list[:sequence_length])
#             y.append(label_list[:sequence_length])
#             text_list = text_list[sequence_length:]
#             label_list = label_list[sequence_length:]
#         x.append(text_list)
        y.append(label_list)
    return [x, y]

In [4]:
x1, y1 = txt_to_list('../datasets/stage1/SampleData_deid.txt', sequence_length)

In [5]:
x2, y2 = txt_to_list('../datasets/stage2/train_1_update.txt', sequence_length)

In [6]:
x4, y4 = txt_to_list('../datasets/stage4/train_2.txt', sequence_length)

In [7]:
x = x1 + x2 + x4
len(x)

76508

In [8]:
y = y1 + y2 + y4
len(y)

76508

In [9]:
# print(x[0])

In [10]:
# print(y[0])

In [11]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
print(len(train_x), len(train_y))
print(len(valid_x), len(valid_y))
print(len(test_x), len(test_y))

48964 48964
12242 12242
15302 15302


In [12]:
# import kashgari
# kashgari.config.use_cudnn_cell = True
# from kashgari.tasks.labeling.models import BiLSTM_Model

# model = BiLSTM_Model()
# model.fit(train_x,
#           train_y,
#           valid_x,
#           valid_y,
#           epochs=10,
#          )
# model.evaluate(test_x, test_y)
# model.save('model/BiLSTM')

In [13]:
# loaded_model = kashgari.utils.load_model('model/BiLSTM')
# loaded_model.predict(test_x)[0]

In [14]:
# import kashgari
# from kashgari.tasks.labeling import BiLSTM_Model
# from kashgari.embeddings import BERTEmbedding

# bert_embed = BERTEmbedding('chinese_L-12_H-768_A-12',
#                            task=kashgari.LABELING,
#                            sequence_length=32,
#                           )

# model = BiLSTM_Model(bert_embed)
# model = BiLSTM_Model()
# model.fit(train_x,
#           train_y,
#           valid_x,
#           valid_y,
#           epochs=10,
#          )

# model.evaluate(test_x, test_y)
# model.save('model/BiLSTM_Model')

In [ ]:
import kashgari
from kashgari.tasks.labeling.models import BiLSTM_CRF_Model
from kashgari.embeddings import BERTEmbedding
kashgari.config.use_cudnn_cell = True

bert_embed = BERTEmbedding('chinese_L-12_H-768_A-12',
                           task=kashgari.LABELING,
                           sequence_length=sequence_length,
                          )

model = BiLSTM_CRF_Model(bert_embed)
model = BiLSTM_CRF_Model()
model.fit(train_x,
          train_y,
          valid_x,
          valid_y,
          epochs=20,
         )

model.evaluate(test_x, test_y)
model.save('model/BiLSTM_CRF_Model')

╭─────────────────────────────────────────────────────────────────────────╮
│ ◎ ○ ○ ░░░░░░░░░░░░░░░░░░░░░  Important Message  ░░░░░░░░░░░░░░░░░░░░░░░░│
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│              We renamed again for consistency and clarity.              │
│                   From now on, it is all `kashgari`.                    │
│  Changelog: https://github.com/BrikerMan/Kashgari/releases/tag/v1.0.0   │
│                                                                         │
│         | Backend          | pypi version   | desc           |          │
│         | ---------------- | -------------- | -------------- |          │
│         | TensorFlow 2.x   | kashgari 2.x.x | coming soon    |          │
│         | TensorFlow 1.14+ | kashgari 1.x.x |                |          │
│         | Keras            | kashgari 0.x.x | legacy version |          │
│           

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 19)]              0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 19, 100)           173800    
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 19, 256)           234496    
_________________________________________________________________
layer_dense (Dense)          (None, 19, 64)            16448     
_________________________________________________________________
layer_crf_dense (Dense)      (None, 19, 31)            2015      
_________________________________________________________________
layer_crf (CRF)              (None, 19, 31)            961       
Total params: 427,720
Trainable params: 427,720
Non-trainable params: 0
_____________________________________________________

In [ ]:
model = kashgari.utils.load_model('model/BiLSTM_CRF_Model')

In [ ]:
import pandas as pd
df = pd.read_csv('../datasets/stage4/development_2.csv')

In [ ]:
def predict_NER(model, text, sequence_length):
    x_list = list()
    while len(text) > sequence_length:
        x_list.append([i for i in text[:sequence_length]])
        text = text[sequence_length:]        
    x_list.append([i for i in text[:sequence_length]])
    y_list = model.predict(x_list)
    merge_list = list()
    for i in y_list:
        for j in i:
            merge_list.append(j)
    return merge_list   

In [ ]:
def output_NER(article_id, text, y):
    output = str()
    flag = False
    for i, j in enumerate(y):
        if j != 'O':
            if j[0] == 'B':
                start_position = i
                entity_type = j.split('-')[-1]
                flag = True
        elif j == 'O' and flag == True:
            end_position = i
            flag = False
            entity_text = text[start_position: end_position]
            output += '{}\t{}\t{}\t{}\t{}\n'.format(article_id, start_position, end_position, entity_text, entity_type)  
    return output

In [ ]:
output="article_id\tstart_position\tend_position\tentity_text\tentity_type\n"
for article_id, text in zip(df['article_id'], df['text']):
    y = predict_NER(model, text, sequence_length)
    output += output_NER(article_id, text, y)    

In [ ]:
print(output)

In [ ]:
output_path='BiLSTM_CRF_Model_16.tsv'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(output)